<h2> Космический корабль "Титаник" <a class="tocSkip"> </h2>
    
Космический корабль "Титаник" был межзвездным пассажирским лайнером, запущенным месяц назад. С почти 13 000 пассажиров на борту, судно отлетело в свое первое плавание, перевозя эмигрантов из нашей Солнечной системы на три новые обитаемые экзопланеты, вращающиеся вокруг близлежащих звезд.

Во время обхода Альфа Центавра по пути к своему первому месту назначения - жаркому 55 Cancri E - неосвзойденный космический корабль Титаник сразился с пространствено-временной аномалией, скрытой в пылевом облаке. К сожалению, он встретил ту же судьбу, что и его тезка 1000 лет назад. Хотя корабль остался нетронутым, почти половина пассажиров была перевезена в другое измерение!
    
Чтобы помочь спасательным экипажам и вернуть потерянных пассажиров, нужно в этом соревновании предсказать, был ли пассажир перевезен в альтернативное измерение во время столкновения космического корабля «Титаник» с аномалией пространства-времени. Чтобы сделать эти прогнозы, нам предоставляется набор личных записей, извлеченных из поврежденной компьютерной системы корабля.

    
<h2> Цель работы  <a class="tocSkip"> </h2>
    
Предсказать, какие пассажиры перевозятся в альтернативное измерение и предсказать стоимость Transported для пассажиров.

<h2> Описания файлов и полей данных  <a class="tocSkip"> </h2>
    
- train.csv - Личные записи примерно для двух третей (~8700) пассажиров, которые будут использоваться в качестве тренировочных данных.
    
    - PassengerId - Уникальный ид для каждого пассажира. Каждый Id принимает форму gggg_pp, где gggg указывает на группу, с которой путешествует пассажир, и pp - это их номер в группе. Люди в группе часто являются членами семьи, но не всегда.
    - HomePlanet - Планета, с которой пассажир уехал, как правило, их планета постоянного проживания.
    - CryoSleep - Указывает, выбран ли пассажир криосон на время рейса. Пассажиры в криосне ограничены в своих каютах.
    - Cabin - Номер салона, в котором остановился пассажир. Принимает форму палубы/номер/боку, где сторона может быть либо P для порта, либо S для старого борта.
    - Destination - Планета, на которую будет доставлен пассажир.
    - Age - Возраст пассажира.
    - VIP - оплатил ли пассажир специальную VIP-услугу во время рейса.
    - RoomService, FoodCourt, ShoppingMall, Spa, VRDeck - Сумма, которую пассажиру выставили за каждый из многочисленных роскошных удобств космического корабля "Титаник".
    - Name - Имя и фамилия пассажира.
    - Transported - Был ли пассажир перевезен в другое измерение. Это цель, столбец, который вы пытаетесь предсказать.
    
    
- test.csv - Личные записи для оставшейся трети (~4300) пассажиров, которые будут использоваться в качестве тестовых данных. Ваша задача - предсказать стоимость Transported для пассажиров в этом наборе.
    
    
- sample_submission.csv - Файл отправки в правильном формате.
    
    - PassengerId- Ид. каждого пассажира в тестовом наборе.
    - Transported- Цель. Для каждого пассажира предскажите либо True, либо False.

In [1]:
!pip install catboost -q

## Изучение данных 
### Загрузка данных 

In [3]:
# Импоритруем библиотеки

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score

# from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier

from sklearn.metrics import accuracy_score


ModuleNotFoundError: No module named 'catboost'

In [ ]:
# Прочитаем файл и запишем в переменную df

df_sample_submission = pd.read_csv('/Users/ilyashirikov/Downloads/Kaggle/spaceship-titanic/sample_submission.csv')
df_test = pd.read_csv('/Users/ilyashirikov/Downloads/Kaggle/spaceship-titanic/test.csv')
df_train = pd.read_csv('/Users/ilyashirikov/Downloads/Kaggle/spaceship-titanic/train.csv')


In [ ]:
# Создадим функцию info_func() для вывода всех необходимых методов показа информации df

def info_func(df):
    display(df.head(10), df.info(), df.describe(), df.isna().sum())

info_func(df_sample_submission)

Отдельно расмотрю т.к. в df_sample_submission с даными все впорядке, их тут не много 4277 в записях нет пропусков, в этой таблице указаны ID всех пасажиров кто остался на карабле "Титаник". С этими данными мы будем сверяться после пердсказания нашей модели. Для удобства False нужно перевести в бинарный вид.

In [ ]:
info_func(df_test)

In [ ]:
info_func(df_train)

В таблице df_test и df_train уже есть что обсудить, df_test имеет так же 4277 строк данных как и в df_sample_submission и в df_train у нас 8693 строк данных, в каждом из признаков есть пропуски не более 3%, разберемся отдельно по признакам:

- PassengerId в обоих таблицах пропусков нет, но было бы удобнее перевести тип данных в INT и разделить этот признак на два признака, где первый признак номер группы, а второй указывал бы на номер пасажира в группе.
- HomePlanet в обоих таблицах есть пропуски, нужно посмотреть уникальные значения и убедиться что Земля и Европа это не одно и тоже. Предположительно пропуски могли взяться из за отсуцтвия постоянного места проживания т.е. пасажир был на этой планете что то типо 'на пересадке'.
- CryoSleep в обоих таблицах есть пропуски, данный признак записан по факту в тип Str, но можно указать его в бинарном виде, с пропусками в данном признаке большие вопросы, тут явно спит пасажир или нет, может пропуски появились из за технических проблем, вот что с ними делать большой вопрос.
- Cabin в обоих таблицах есть пропуски, тут у нас номер салона, в котором остановился пассажир, по данным в основном это семейные пары, возможно было бы верно заполнить пропуски ближайшими соседями или посмотреть признак Name. Этот признак нужно будет кодировать.
- Destination в обоих таблицах есть пропуски и тут по аналогии с Cabin можно взять группу и взять их среднее по месту прибытия и заполнить этими значениями пропуски, вполне возможно что группы состовляли на основе мест прибытия для удобства расположения вы выгрузки таких групп. Этот признак нужно будет кодировать.
- Age этот признак так же с пропусками и визуально через функцию discribe, выбрасы или анамальные значения не выявлены, максимальный возраст 79 лет, минимальный указан ноль, вполне возможно что на карабле новорожденные дети и новые технологии 2912 года способны на рейсы с такими посажырами, средний возраст составляет 29 лет, 75% пасажиров в возрасте до 38 лет.
- VIP в обоих таблицах есть пропуски, мне кажется логично что если в этом признаке у пасажира нет значения True значит что VIP не подключен, предолгаю заменить значения пропусков на False.
- RoomService, FoodCourt, ShoppingMall, Spa, VRDeck так же имеют пропуски, в аналогии с VIP, считаю что если нет суммы уплаченой за сервис, означает что все пропуски равны 0, т.к. пустые значения могут указывать на то что пасажир не пользовался сервисами.
- Name для нас не имеет сильной значимости кроме как на основе него заполнять пропуски в других признаках, при обучении модели этот признак будет неинформативен, смысла заполнять или удалять его пропуски нет.

## Разведочный анализ данных

In [ ]:
plt.pie(df_train.HomePlanet.value_counts(),labels = ['Earth','Europa','Mars'])
plt.title("HomePlanet")
plt.show()